# Cancer Radiology Optimization

This Code Deals with Python Implimentation of the Paper [*Optimizing the Delivery of
Radiation Therapy to Cancer Patients* by David M Sheperd et. al. 1999](https://doi.org/10.1137/S0036144598342032) and then solving the problem given below using the Optimization Technique described in the above paper.

The Radiotherapy technique described in the above paper is also called as ***Tomotherapy***

Suppose we want to design an IMRT setup to obtain a dose corresponding to the following figure.

![Image](https://lh3.googleusercontent.com/pVkc8Or2Tj2ninBZCKhSbRMXABF4YZkXpUdC7gaBrtRSJRX-T8m47gbCuDX-CXcxqMaa7P4yxqU)



Assume that we want to achieve close to 10 units of radiation in the red region (denoting a tumor)
and be as much below 4 units as possible in the remaining portion. You may assume the radiation
intensity diminishes to half every 4 units (which means that this relation is not linear).
Assuming that each beam is 1 unit wide, find a suitable configuration of beam weights and angles
that can realize a plan as close as possible to the desired treatment. You may further assume that
each beam is composed of 20 small beamlets each of whose intensities can be controlled separately.
Solve one of the LPs or QPs proposed described by Shephard et al and find the optimal configuration.
Describe the configuration in a readable manner. Also draw a pictorial representation of
the intensity pattern achieved by your proposed solution. Comment on its quality. Clearly state
all your assumptions and choices of parameters.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from amplpy import AMPL
from scipy.spatial import distance as dist
from math import exp

### The Equation Formulation used is as per the below section of the original paper


![Optimization Equation Formulation](https://lh3.googleusercontent.com/AgAYxUEkWx8GepAaODjgxY03sk7b8TdVTMcDcicAVFZ1AuAyPjeHp_cbyOoXz1hUJMA--xB-bVo)

### Initializing all the parameters and variables

In [5]:
radi     =  3  #Radius of the circular organ region   
grid_sz  =  6  # Length of the grid square area (grid size)
grid_res = 20  # 1 unit length to be devided in grid_res Number of pizels per length
beamlets = 20  # Number of beamlets per beam. All the beamlets in a beam have same angle
pos      =  5  # Total angular positions of the setup

#All length parameters will be multiplied with grid_res from hence forth
grid_sz *= grid_res
radi    *= grid_res
center   = (radi, radi)
positions = np.linspace(0, 360, num=pos, endpoint=False)

***********************************************************************
* Please make sure that the AMPL folder is in the system search path. *
***********************************************************************


RuntimeError: AMPL could not be started. Message from process thread:
cannot execute ampl: No such file or directory



### Function for generating Dij_p matrix

Dij_p matrix is generated for each beamlet at each position. 

In [ ]:
def GET_Dijp(angle, beamlet_no):
    """
    Beamlet Number starts at 0 to n-1 for n beamlets
    Returns a D_ij_p matrix
    """
    def IN_COL(coordinates):
        i = coordinates[0]; j = coordinates[1]
        res = grid_res
        if i>=(2.5+beamlet_no/20)*res and i<=(2.55+beamlet_no/20)*res:
            return(exp(-1*j/(4*grid_res)))
        else:
            return 0

    col = np.asarray([[IN_COL((i, j)) 
                        for i in range(grid_sz)] for j in range(grid_sz)])

    import scipy.ndimage.interpolation as inter
    col = inter.rotate(col, angle, reshape = False)
    return col

### Map Risk zone and Tumor Zone in Matrix form


In [ ]:
def IN_TUMOR(coordinates):
    """
    Uses 6x6 grid cordinates
    Returns in a given 2D coordinate lies within I shaped Tumor
    """
    i = coordinates[0]; j = coordinates[1]
    res = grid_res
    if i>=2.5*res and i<=3.5*res and j>=1*res and j<=5*res:
        return 1
    elif (((i>=1.5*res and i<=2.5*res) or(i>=3.5*res and i<=4.5*res)) and 
          ((j>=1*res and j<=2*res) or (j>=4*res and j<=5*res))):
        return 1
    else:
        return 0

#Circular region (1 if in region, 0 o/w)
map_risk  = np.asarray([[1 if dist.euclidean((i, j), center)<= radi else 0
                        for i in range(grid_sz)] for j in range(grid_sz)])
# I Shaped Tumor Region (1 if in region, 0 o/w)
map_tumor = np.asarray([[IN_TUMOR((i, j)) 
                        for i in range(grid_sz)] for j in range(grid_sz)])
# Risk region without tumor (1 if in region, 0 o/w)
map_risk -= map_tumor

In [ ]:
#Initialization of the optimization model 
problem = AMPL()

problem.eval("""
param theta; #A 3 dimentional theta for set T, R, N
param SETS = 1..3;

set GRID_X;
set GRID_Y;
set BEAMLETS;
set POS;  # Total number of beam angle positions to be used 

#For every positon, for every beamlet p,
#we calculate intensity at every pixel (x, y)
param D_ijp{POS, BEAMLETS, GRID_X, GRID_Y};

var w{POS, BEAMLETS}; #weights for each beamlet in each position
var D_ij{POS, GRID_X, GRID_Y};

#A 3 channel T, R, N set's 1-0 mapping
#for a channel, say R(represented by 1),
#X, Y = 1 if (x, y) belongs to set R
var MAP_TRN{3, GRID_X, GRID_Y}; 


minimize objective : sum{trn in 1..3}theta[trn]*sum{i in 1..GRID_X, j in 1..GRID_Y} MAP_TRN[trn, i, j]*sum{p in 1..POS}D_ij[p, i, j]

subject to con{i in 1..GRID_X, j in 1..GRID_Y, p in 1..POS}: D_ij[p, i, j] = sum{b in 1..BEAMLETS}D_ijp[p, b, i, j]

""")

## Optimization Code Starts

### optimization Parameters

In [ ]:
theta = []